In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_elon_musk = pd.read_csv('/kaggle/input/all-elon-musks-tweets/TweetsElonMusk.csv', parse_dates=True)
df_elon_musk.head()

In [ ]:
df_elon_musk.info()

# remove 100 null columes

In [ ]:
null_col = df_elon_musk.isna().all(axis=0)
null_col = list(null_col[null_col==True].index)
null_col

In [ ]:
df_elon_musk_1 = df_elon_musk.drop(null_col, axis=1)
df_elon_musk_1.columns

In [ ]:
df_elon_musk_1.head()

# drop one value

In [ ]:
df_elon_musk_2 = df_elon_musk_1.drop(['timezone','user_id','username', 'retweet'], axis=1)

In [ ]:
useful_col = ['date', 'time', 'tweet', 'replies_count', 'retweets_count', 'likes_count']
useful_to_map_with_crypto = df_elon_musk_2[useful_col]

# Extract keyword from tweet and count

In [ ]:
tweet = useful_to_map_with_crypto['tweet']

In [ ]:
# make text lower character all
tweet = tweet.str.lower()

In [ ]:
tweet.str.contains('coin').value_counts()

In [ ]:
useful_to_map_with_crypto['mention_coin'] = tweet.str.contains('coin')
useful_to_map_with_crypto.sample(5)

In [ ]:
useful_to_map_with_crypto['mention_coin'].value_counts()

In [ ]:
useful_to_map_with_crypto['mention_bitcoin'] = tweet.str.contains('bitcoin')
useful_to_map_with_crypto['mention_bitcoin'].value_counts()

In [ ]:
tweet.str.contains('crypto').value_counts()

In [ ]:
useful_to_map_with_crypto['mention_crypto'] = tweet.str.contains('crypto')
useful_to_map_with_crypto['mention_crypto'].value_counts()

In [ ]:
# save data
useful_to_map_with_crypto.to_csv('elon_tweet_clean1.csv')

In [ ]:
useful_to_map_with_crypto['date'] = pd.to_datetime(useful_to_map_with_crypto['date'], format="%Y/%m/%d")
useful_to_map_with_crypto.sample(5)

# simple plot

In [ ]:
useful_to_map_with_crypto['date_parsed'].dtypes

In [ ]:
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime

In [ ]:
dates = useful_to_map_with_crypto.groupby('date_parsed')['mention_coin'].sum()

In [ ]:
plt.figure(figsize=(14,6))
plt.title("trend that Elon Musk tweeted mention 'coin'")
sns.lineplot(data=dates.loc['2021-01-01':])

## crypto fast cleansing

In [ ]:
def fast_clean(df):
    df.drop(columns=['Date','Vol.'], inplace=True)
    df.rename(columns={'Date1':'month','Vol.change%':'VolChange_Crypto','Change %':'Change_Crypto','Vol.change(DowJones)':'VolChange_DownJones','Change %(DowJones)':'Change_DownJones','Vol.change(NASDAQ)':'VolChange_NASDAQ','Change %(NASDAQ)':'Change_NASDAQ','Vol.change(Gold)':'VolChange_Gold','Change %(Gold)':'Change_Gold','Vol.change(crude oil)':'VolChange_crudeoil','Change %(crude oil)':'Change_crudeoil','Change %(EUR_USD)':'Change_EUR_USD','Vol.change(NVDA)':'VolChange_NVDA','Change %(NVDA)':'Change_NVDA','Vol.change(AMD)':'VolChange_AMD','Change %(AMD)':'Change_AMD','Vol.change(PA=F)':'VolChange_PA=F','Change %(PA=F)':'Change_PA=F'}, inplace=True)
    df.dropna(inplace=True)
    df['Volumn'] = pd.to_numeric(df.Price, errors='coerce')
    df['VolChange_Crypto'] = pd.to_numeric(df.Open, errors='coerce')
    df['VolChange_Gold'] = pd.to_numeric(df.High, errors='coerce')
    df['Change_Gold'] = pd.to_numeric(df.Low, errors='coerce')
    df['VolChange_crudeoil'] = pd.to_numeric(df.Volumn, errors='coerce')
    df['Change_crudeoil'] = pd.to_numeric(df.Change_Crypto, errors='coerce')
    df['VolChange_PA=F'] = pd.to_numeric(df.VolChange_Crypto, errors='coerce')
    df['Change_PA=F'] = pd.to_numeric(df.VolChange_DownJones, errors='coerce')
    df['Datenew'] = pd.to_datetime(df['Datenew'], format="%d/%m/%Y")
    return df

In [ ]:
df_crypto = pd.read_csv('/kaggle/input/private-crypto2/Cryptodata2.csv', parse_dates=True)
df_crypto.head()

In [ ]:
crypto = fast_clean(df_crypto)
crypto.head()

In [ ]:
def plot_compare(df, crypto):
    units = [['Price_DowJones','Price_GOLD', 'Price_NASDAQ', 'Price_NVDA'], ['Price_EUR_USD', 'Price_crude oil', 'Price_PA=F', 'Price_AMD']]
    coin = df[df['Cryptoname']==crypto]

    fig, ax1 = plt.subplots(2, 4, figsize=(20,8))
    fig.autofmt_xdate()
    for i in range(2):
        for j in range(4):
            color = 'tab:red'
            ax1[i, j].set_xlabel('Date')
            ax1[i, j].set_ylabel(crypto+' price', color=color)
            ax1[i, j].set_title(crypto+' price and '+units[i][j]+' in 2021')
            ax1[i, j].fmt_xdata = mdates.DateFormatter('%m-%d')
            ax1[i, j].tick_params(axis='y', labelcolor=color)
            sns.lineplot(data=coin, ax=ax1[i, j], x='Datenew', y='Price', color=color)

            ax2 = ax1[i, j].twinx()  # instantiate a second axes that shares the same x-axis

            color = 'tab:blue'
            ax2.set_ylabel(units[i][j], color=color)  # we already handled the x-label with ax1
            ax2.tick_params(axis='y', labelcolor=color)
            sns.lineplot(data=coin, ax=ax2, x='Datenew', y=units[i][j], color=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()

In [ ]:
crypto.sample()

In [ ]:
crypto_col = ['month', 'monthnum', 'daynum', 'yearnum', 'Datenew', 'Price',
              'Cryptoname', 'Price_DowJones', 'Price_NASDAQ', 'Price_GOLD',
              'Price_crude oil', 'Price_EUR_USD', 'Price_NVDA', 'Price_AMD', 'Price_PA=F']
main_crypto = crypto[crypto_col]
main_crypto.sample()

In [ ]:
elon_crypto = main_crypto.set_index('Datenew').join(useful_to_map_with_crypto.set_index('date_parsed'), how='inner')
elon_crypto.head()

In [ ]:
elon_crypto[elon_crypto['mention_coin']]